In [1]:
import torch
import torch.nn as nn

In [2]:
inputs = torch.Tensor(1, 1, 28, 28)
inputs

tensor([[[[ 0.0000e+00,  0.0000e+00,  3.8983e-39,  3.3329e-29,  0.0000e+00,
            0.0000e+00,  3.4391e-12,  2.5353e+30,  6.3058e-44,  1.6661e-41,
            1.4035e+02,  6.1608e+01,  6.0115e+07,  4.5200e+30,  2.8026e-45,
            1.7020e-41,  1.1429e-39,  8.8039e-10,  2.5353e+30,  6.8664e-44,
            1.8096e-41,  4.7245e-39,  2.9829e-30,  2.5353e+30,  9.1084e-44,
            1.8814e-41,  4.9082e-39,  7.6362e-28],
          [ 2.5353e+30,  2.8026e-44,  7.6931e-42,  4.8163e-39,  1.2398e-38,
            1.9629e-25,  0.0000e+00,  9.4832e-38,  1.5166e-36,  2.4267e-35,
            2.4267e-35,  3.8368e-31,  1.4948e+02,  2.6460e+11,  4.9662e-08,
            4.5592e+30,  2.6378e+23,  7.1443e+31,  2.8450e-36,  2.5353e+30,
            1.4013e-45,  1.5947e-42,  4.0824e-40,  4.1991e+30,  7.0065e-45,
            5.7711e-05,  2.5353e+30,  7.9874e-44],
          [ 2.0966e-41,  5.4592e-39,  5.1245e-20,  2.5353e+30,  6.1657e-44,
            1.6303e-41,  6.0102e-39,  1.4771e-02,  2.5353e+30,

In [3]:
inputs.shape

torch.Size([1, 1, 28, 28])

In [4]:
conv1 = nn.Conv2d(1, 32, 3, padding=1)
pool = nn.MaxPool2d(2)
pool

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)

In [5]:
conv2 = nn.Conv2d(32, 64, 3, padding=1)
conv2

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

In [6]:
pool

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)

In [7]:
out = conv1(inputs)
out.shape

torch.Size([1, 32, 28, 28])

In [8]:
out = pool(out)
out.shape

torch.Size([1, 32, 14, 14])

In [9]:
out = conv2(out)
out.shape

torch.Size([1, 64, 14, 14])

In [10]:
out = pool(out)
out.shape

torch.Size([1, 64, 7, 7])

In [11]:
out.size(0)

1

In [12]:
out.size(1)

64

In [13]:
out.size(2)

7

In [14]:
out.size(3)

7

In [15]:
out = out.view(out.size(0), -1)
out.shape

torch.Size([1, 3136])

In [16]:
fc = nn.Linear(3136, 10)
fc(out)

tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]],
       grad_fn=<AddmmBackward0>)

In [17]:
out.shape

torch.Size([1, 3136])

## EX

In [18]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [19]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [29]:
torch.cuda.is_available()

False

In [20]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [36]:
# MNIST dataset

mnist_train = dsets.MNIST(root='MNIST_data/',
                         train=True,
                         transform=transforms.ToTensor(),
                         download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [37]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                         batch_size=batch_size,
                                         shuffle=True,
                                         drop_last=True)

In [43]:
class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc1 = nn.Linear(3*3*128, 625, bias=True)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [44]:
model = CNN().to(device)
value = torch.Tensor(1, 1, 28, 28).to(device)
print((model(value)).shape)

torch.Size([1, 10])


In [45]:
model

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=1152, out_features=625, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=625, out_features=10, bias=True)
)

In [46]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [47]:
# training
total_batch = len(data_loader)

for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost/total_batch
        
    print('[Epoch:{}] cost = {}'.format(epoch+1, avg_cost))
    
print('Learning Finished!')

[Epoch:1] cost = 0.16390322148799896
[Epoch:2] cost = 0.03996256738901138
[Epoch:3] cost = 0.028612811118364334
[Epoch:4] cost = 0.020586198195815086
[Epoch:5] cost = 0.017301669344305992
[Epoch:6] cost = 0.014706585556268692
[Epoch:7] cost = 0.010303092189133167
[Epoch:8] cost = 0.010482361540198326
[Epoch:9] cost = 0.010244425386190414
[Epoch:10] cost = 0.00881370808929205
[Epoch:11] cost = 0.006220994051545858
[Epoch:12] cost = 0.007970891892910004
[Epoch:13] cost = 0.005872415844351053
[Epoch:14] cost = 0.0051016779616475105
[Epoch:15] cost = 0.004975962918251753
Learning Finished!


In [48]:
# Test
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.9886000156402588
